# Mini Exercises: Large Language Models with Hugging Face

Welcome to a quick tour of practical activities you can use to help students experiment with open-source large language models (LLMs). This notebook is structured as a mini workshop with short, self-contained tasks that demonstrate different natural language capabilities.

## Learning goals
- Understand how to load and run Hugging Face Transformers pipelines for common LLM tasks.
- Practice prompt engineering for short-form text generation.
- Explore how encoder-decoder models perform abstractive summarization.
- Experiment with zero-shot classification to tag text with arbitrary labels.


## 0. Setup
These exercises rely on the [transformers](https://huggingface.co/docs/transformers/index) library and a few lightweight models hosted on Hugging Face.

Run the cell below to install any missing dependencies inside your environment. (This only needs to be done once per runtime.)

In [ ]:
# If running on a fresh environment, uncomment the next line.
# !pip install -q transformers datasets accelerate torch --upgrade


You can verify your installation and the available devices (CPU/GPU) with the following optional diagnostic cell.

In [ ]:
# Optional: check library versions and available hardware
from transformers.utils import is_torch_available

if is_torch_available():
    import torch
    import transformers
    print('Transformers version:', transformers.__version__)
    print('Torch version:', torch.__version__)
    print('Available device:', 'cuda' if torch.cuda.is_available() else 'cpu')
else:
    print('Transformers is not installed yet. Run the pip install cell above.')


---

## 1. Prompt-based text generation
Large language models trained for causal language modeling (CLM) can generate coherent continuations of a prompt. We will use [`distilgpt2`](https://huggingface.co/distilgpt2), a lightweight distilled GPT-2 variant suitable for classroom demos.

### Exercise
1. Choose a short topic sentence to seed the model.
2. Experiment with parameters such as `max_new_tokens`, `temperature`, and `num_return_sequences`.
3. Compare the generated completions and discuss how sampling parameters affect creativity vs. coherence.

In [ ]:
from transformers import pipeline

# TODO: replace this prompt with your own idea (e.g., a course topic or debate question).
prompt = "In 2030, classrooms will use AI to"

try:
    generator = pipeline("text-generation", model="distilgpt2")
    outputs = generator(
        prompt,
        max_new_tokens=60,
        temperature=0.8,
        num_return_sequences=2,
        do_sample=True,
    )

    for idx, output in enumerate(outputs, start=1):
        print(f"--- Completion {idx} ---")
        print(output["generated_text"])
        print()
except OSError as exc:
    print("⚠️ Unable to load the text-generation pipeline:", exc)
    print("If you're working offline, download 'distilgpt2' ahead of time or point HF_HOME to a cache directory.")


**Discussion prompts**
- How does increasing the temperature change the tone of the completions?
- What guardrails might you need before deploying an open-ended generator to students?
- Which settings produce outputs closest to the voice you want?


---

## 2. Abstractive summarization
Encoder-decoder transformer models excel at condensing long passages. Here, you will summarize an education article using [`sshleifer/distilbart-cnn-12-6`](https://huggingface.co/sshleifer/distilbart-cnn-12-6).

### Exercise
1. Paste a paragraph (3-5 sentences) from an educational news article or research abstract.
2. Run the summarization pipeline and inspect the result.
3. Tweak `max_length` and `min_length` to balance brevity and completeness.

In [ ]:
from transformers import pipeline

try:
    summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

    # TODO: replace this article text with one you want to summarize.
    article = """Artificial intelligence tools are becoming more common in classrooms, helping teachers personalize assignments and
provide instant feedback. Researchers caution that educators should combine AI suggestions with their professional judgment to ensure equitable learning experiences."""

    summary = summarizer(
        article,
        max_length=80,
        min_length=30,
        do_sample=False,
    )[0]["summary_text"]
    print(summary)
except OSError as exc:
    print("⚠️ Unable to load the summarization pipeline:", exc)
    print("Download 'sshleifer/distilbart-cnn-12-6' in advance or provide a local cache path when working offline.")


**Discussion prompts**
- Does the model capture the main claim accurately?
- What important nuance (if any) was lost in the summarization?
- How might students misuse a summarization tool, and how could you guide them?


---

## 3. Zero-shot topic tagging
Zero-shot classification lets you assign custom labels to a piece of text without additional fine-tuning. We will use [`facebook/bart-large-mnli`](https://huggingface.co/facebook/bart-large-mnli), which scores how well each label matches the text.

### Exercise
1. Create a list of 3-5 candidate labels relevant to your subject area (e.g., *assessment*, *student wellbeing*, *policy*).
2. Supply a short passage (2-4 sentences) describing an educational scenario.
3. Interpret the confidence scores returned by the model and discuss how you might use them to triage responses.

In [ ]:
from transformers import pipeline

try:
    classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

    # TODO: customize the text and labels for your own scenario.
    scenario = """The district is piloting a blended learning program where students rotate between online practice and small-group tutoring. Administrators are tracking engagement data to see which approach improves math outcomes."""
    candidate_labels = ["personalization", "assessment", "teacher training"]

    result = classifier(scenario, candidate_labels=candidate_labels, multi_label=True)
    for label, score in zip(result["labels"], result["scores"]):
        print(f"{label:>20}: {score:.2f}")
except OSError as exc:
    print("⚠️ Unable to load the zero-shot classifier:", exc)
    print("Download 'facebook/bart-large-mnli' ahead of time or configure a local Hugging Face cache for offline work.")


**Discussion prompts**
- Which labels does the model consider most relevant, and why?
- How sensitive are the results to the wording of your labels?
- What ethical considerations arise when automatically tagging student submissions?


---

## 4. Extension ideas
Looking for additional challenges? Try one of the following mini-projects:

- **Benchmark two models** on the same task (e.g., compare `distilgpt2` vs. `tiiuae/falcon-7b-instruct` using the [text-generation-inference](https://huggingface.co/docs/text-generation-inference/index) API).
- **Collect student reflections** by prompting them to critique the generated outputs and propose rubric criteria.
- **Build a simple user interface** with [Gradio](https://gradio.app/) so students can test prompts live.

Feel free to adapt, expand, or remix these exercises for your classroom!
